In [20]:
import pandas as pd
import numpy as np

# Read the CSV files
file1 = '../processed/processed_consumer_spending.csv'  # 2006-2012
file2 = '../processed/processed_consumer_spending2.csv'  # 2013-2020

# Load the data, making sure to treat commas in numbers correctly
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Data cleaning for df1
# Replace any non-numeric values with NaN for year columns
year_columns1 = [str(year) for year in range(2006, 2013)]
for col in year_columns1:
    if col in df1.columns:
        df1[col] = pd.to_numeric(df1[col], errors='coerce')

# Data cleaning for df2
# Clean the 2020 column which contains dollar signs and commas
if '$61,334' in df2.columns.values:
    # Rename the column to '2020'
    df2 = df2.rename(columns={'$61,334': '2020'})

# Clean all numeric columns in df2
year_columns2 = [str(year) for year in range(2013, 2021)]
for col in year_columns2:
    if col in df2.columns:
        # Remove $ and , from strings before converting to numeric
        if df2[col].dtype == object:
            df2[col] = df2[col].astype(str).str.replace('$', '', regex=False)
            df2[col] = df2[col].str.replace(',', '', regex=False)
        df2[col] = pd.to_numeric(df2[col], errors='coerce')

# Merge the dataframes on 'Item' column
merged_df = pd.merge(df1, df2, on='Item', how='outer', suffixes=('_1', '_2'))

# If there are duplicate 'Category' columns, keep one
if 'Category_1' in merged_df.columns and 'Category_2' in merged_df.columns:
    # Fill NaN values in Category_1 with values from Category_2
    merged_df['Category'] = merged_df['Category_1'].fillna(merged_df['Category_2'])
    merged_df = merged_df.drop(['Category_1', 'Category_2'], axis=1)
elif 'Category_1' in merged_df.columns:
    merged_df = merged_df.rename(columns={'Category_1': 'Category'})
elif 'Category_2' in merged_df.columns:
    merged_df = merged_df.rename(columns={'Category_2': 'Category'})

# Create a list of all year columns in the correct order
all_years = [str(year) for year in range(2006, 2021)]
year_cols = [col for col in all_years if col in merged_df.columns]

# Reorder columns with 'Item' first, followed by years, then 'Category'
columns_order = ['Item'] + year_cols + ['Category']
# Filter to only include columns that actually exist in the merged dataframe
columns_order = [col for col in columns_order if col in merged_df.columns]
merged_df = merged_df[columns_order]

# Save the merged dataframe to a new CSV file
merged_df.to_csv('merged_consumer_spending_2006_2020.csv', index=False)

print(f"Successfully merged data! Output saved to merged_consumer_spending_2006_2020.csv")
print(f"The merged dataset has {merged_df.shape[0]} rows and {merged_df.shape[1]} columns.")
print(f"Year columns included: {', '.join(year_cols)}")

Successfully merged data! Output saved to merged_consumer_spending_2006_2020.csv
The merged dataset has 199 rows and 17 columns.
Year columns included: 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020


In [22]:
pd.read_csv('../processed/processed_consumer_spending.csv')

,Item,2006,2007,2008,2009,2010,2011,2012,Category
0,Average Annual Expenditure,48400.0,49638.0,50486,49067.0,48109,49705.0,51442.0,Average Annual Expenditure
1,Food,6111.0,6133.0,6443,6372.0,6129,6458.0,6599.0,Food
2,Food at home,3417.0,3465.0,3744,3753.0,3624,3838.0,3921.0,Food at home
3,Cereals and bakery products,446.0,460.0,507,506.0,502,531.0,538.0,Cereals and bakery products
4,Cereals and cereal products,143.0,143.0,170,173.0,165,175.0,182.0,Cereals and cereal products
...,...,...,...,...,...,...,...,...,...
141,Other entertainment,52.0,68.0,59,57.0,58,67.0,54.0,Other entertainment
142,Personal care products and services d/,16.0,18.0,12,12.0,13,15.0,16.0,Personal care products and services d/
143,Reading d/,1.0,1.0,1,1.0,1,2.0,2.0,Reading d/
144,Education,210.0,283.0,324,229.0,221,216.0,260.0,Education
